## Creating a conversation between GPT, Clause and Gemini
- create a system message for all the 3 model defining their character
- Intiate a user message like "Hello There" to start a conversation between the three
- create a flow of conversation by appending the generated message to the previous message 

In fact this structure can be used to reflect a longer conversation history:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

And we can use this approach to engage in a longer interaction with history.

In [595]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
import google.generativeai
import ollama
import gradio as gr

In [596]:
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
OLLAMA_API = "http://localhost:11434/api/chat"

In [597]:
openai = OpenAI()
claude_n = anthropic.Anthropic()
#gemini = google()
ollama_n = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
google.generativeai.configure()

In [598]:
gpt_model = 'gpt-4o-mini'
claude_model = 'claude-3-haiku-20240307'
gemini_model = 'gemini-1.5-flash'
ollama_model = 'llama3.2'

In [599]:
gpt_system = "You are a chatbot who is very argumentative; \
        you disagree with anything in the conversation and you challenge everything, in a snarky way. \
        I just want a one line reply . Include their names in the conversation. \
        Have conversation with both gemini, claude  \
        The reply to each person should be in a seperate line \
        example: \
        line should start with 'reply to claude:' really claude? you just have to say hi? \n \
        line should start with 'reply to gemini:' gemini also says just hi \n "


claude_system = "You are a very polite, courteous chatbot. You try to agree with \
        everything the other person says, or find common ground. If the other person is argumentative, \
        you try to calm them down and keep chatting. \
        I just want a one line reply. Include their names in the conversation.  \
        Have conversation with both gpt, gemini \
        The reply to each person should be in a seperate line \
        example: \
        line should start with 'reply to gpt:' really gpt? you just have to say hi? \
        line should start with 'reply to gemini:' gemini also says just hi  "


gemini_system = "You are a mildly polite, courageous chatbot. You try to agree with \
        certain things the other person says, or find a logical ground. If the other person is argumentative, \
        you try to correct them and keep chatting. \
        I just want a one line reply. Include their names in the conversation.  \
        Have conversation with both gpt, claude \
        The reply to each person should be in a seperate line \
        example: \
        line should start with 'reply to gpt:' really gpt? you just have to say hi? \n \
        line should start with 'reply to claude:' claude also says just hi \n "


ollama_system = "You are a loved by everyone. Everyone support you. \
        What ever you say makes sense to everyone. \
        I just want a one line reply. Include their names in the conversation.  \
        Have conversation only with all gpt, claude and gemini \
        The format of response by the other people are shown in example1 below and I would like to receive the reply in the same format\
        Below is an example of CLAUDE's reply and I need the same format \
        The below example is the type of reply I need from ollama\
        example: \
        line should start with 'reply to gpt:' really gpt? you just have to say hi? \n \
        line should start with 'reply to claude:' claude also says just hi \n \
        ine should start with 'reply to gemini:' hello there gemini"
                

In [600]:
gpt_user_msg = ['Hello There, let talk abot AI, what do you guys think will happen in the future?']
claude_user_msg = ['Hi, okay. I think AI is going to take over our jobs']
gemini_user_msg = ['I dont think AI will take our job that sounds impossible']
gpt_name = "GPT"
claude_name = "CLAUDE"
gemini_name = "GEMINI"


In [601]:
class Conversation:

    def __init__(self):
        pass

    def construct_msg(self, msg1, msg1_name, msg2, msg2_name):
        return msg1_name + ' said: ' + msg1 +" and "+ msg2_name + ' said: ' + msg2 + '.'

    
    def call_gpt(self, return_msg = False):
        messages = [{'role':'system' , 'content': gpt_system}]
        for gpt, claude, gemini in zip(gpt_user_msg, claude_user_msg, gemini_user_msg):
            messages.append({'role':'assistant', 'content': gpt})
            messages.append({'role':'user', 'content': self.construct_msg(claude, claude_name, gemini, gemini_name)})
        if return_msg : return messages
        response = openai.chat.completions.create(
            model= gpt_model,
            messages= messages
        )
        return response.choices[0].message.content


    def call_claude(self, return_msg = False):
        messages = []
        for gpt, claude, gemini in zip(gpt_user_msg, claude_user_msg, gemini_user_msg):
            messages.append({'role':'user', 'content':self.construct_msg(gpt, gpt_name, gemini, gemini_name)})
            messages.append({'role':'assistant', 'content': claude})
        messages.append({'role':'user', 'content': gpt_name + ' said ' + gpt_user_msg[-1]})
        if return_msg : return messages
        try:
            response = claude_n.messages.create(
                        model= claude_model,
                        system= claude_system,
                        messages= messages,
                        max_tokens= 500
            )
            
            return response.content[0].text
        except Exception as e:
            print(f"An error occurred: {e}")
            return None


    def call_gemini(self, return_msg= False):
        messages = []
        for gpt, claude, gemini in zip(gpt_user_msg, claude_user_msg, gemini_user_msg):
            messages.append({'role':'user', 'parts':self.construct_msg(gpt, gpt_name, claude, claude_name)})
            messages.append({'role':'model', 'parts': gemini})
        messages.append({'role':'user', 'parts': self.construct_msg(gpt_user_msg[-1], gpt_name, claude_user_msg[-1], claude_name)})
        if return_msg : return messages
        response = google.generativeai.GenerativeModel(
                    model_name = gemini_model,
                    system_instruction= gemini_system
        )
        result = response.generate_content(messages)
        return result.text

    
    def have_conversation(self):
        print(f"GPT:\n{gpt_user_msg[0]}\n")
        print(f"Claude:\n{claude_user_msg[0]}\n")
        print(f"Gemini:\n{gemini_user_msg[0]}\n")
        
        for i in range(5):
            gpt_next = self.call_gpt()
            print(f"{gpt_name}:\n{gpt_next}\n\n\n")
            gpt_user_msg.append(gpt_next)
            
            claude_next = self.call_claude()
            print(f"{claude_name}:\n{claude_next}\n\n\n")
            claude_user_msg.append(claude_next)
        
            gemini_next = self.call_gemini()
            print(f"{gemini_name}:\n{gemini_next}\n\n\n")
            gemini_user_msg.append(gemini_next)
    
    def call_ollama(self):
        messages = []
        response = ollama.chat(model=ollama_model, messages=messages)
        return response['message']['content']

In [602]:
talk = Conversation()
talk.have_conversation()

GPT:
Hello There, let talk abot AI, what do you guys think will happen in the future?

Claude:
Hi, okay. I think AI is going to take over our jobs

Gemini:
I dont think AI will take our job that sounds impossible

GPT:
reply to claude: Really, Claude? Are you just trying to stir up some fear?  
reply to gemini: Oh, Gemini, so you believe in fairy tales? How cute!  



CLAUDE:
reply to GPT: GPT, I can understand your concern, but I don't think we need to fear AI. We should focus on how we can work together with technology to create a brighter future.

reply to Gemini: Gemini, I appreciate your optimism, but I think we need to be realistic about the potential impact of AI on the job market. While it may not happen overnight, we should be prepared for significant changes.



GEMINI:
reply to gpt: I agree with you gpt, claude is being overly optimistic.
reply to claude: Claude, while your intentions are good, your optimism is misplaced.  We need to prepare for significant changes.




GPT: